In [1]:
import pandas as pd
import nfl_data_py as nfl

In [2]:
stat_cols = ['completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr',]

In [3]:
weekly = pd.read_parquet('data/weekly.parquet')
per_game = weekly.groupby(['season','week','recent_team', 'opponent_team'])[stat_cols].sum().reset_index().sort_values(by=['season','week'])
per_game['pass_pct'] = per_game['attempts'] / (per_game['attempts'] + per_game['carries'])

In [4]:
cols2roll = ["completions", "attempts", "carries", "passing_yards", "rushing_yards", "pass_pct", "receiving_yards"]

In [46]:
roll_stats = [x + '_team_roll' for x in cols2roll]
print(roll_stats)
team = per_game.copy()
team[roll_stats] = team.groupby("recent_team")[
    cols2roll
].transform(lambda x: x.rolling(7, min_periods=1).mean()).sort_values(by='completions')
team[roll_stats] = team[roll_stats] / team[roll_stats].mean()
team

['completions_team_roll', 'attempts_team_roll', 'carries_team_roll', 'passing_yards_team_roll', 'rushing_yards_team_roll', 'pass_pct_team_roll', 'receiving_yards_team_roll']


,season,week,recent_team,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,fantasy_points,fantasy_points_ppr,pass_pct,completions_team_roll,attempts_team_roll,carries_team_roll,passing_yards_team_roll,rushing_yards_team_roll,pass_pct_team_roll,receiving_yards_team_roll
0,2010,1,ARI,LA,22,41,297.0,1,0.0,2.0,...,60.779999,82.779999,0.661290,1.029529,1.208136,0.799462,1.225095,0.999984,1.173903,1.225108
1,2010,1,ATL,PIT,27,44,252.0,0,1.0,2.0,...,39.080002,66.080002,0.637681,1.263513,1.296536,0.951740,1.039475,0.517849,1.131993,1.039486
2,2010,1,BAL,NYJ,20,38,248.0,0,1.0,2.0,...,39.619999,59.619999,0.520548,0.935936,1.119735,1.332436,1.022975,0.437493,0.924061,1.022986
3,2010,1,BUF,MIA,18,34,139.0,1,0.0,3.0,...,34.459999,52.459999,0.666667,0.842342,1.001869,0.647183,0.573361,0.446421,1.183447,0.573367
4,2010,1,CAR,NYG,14,35,182.0,1,3.0,4.0,...,36.380001,50.380001,0.593220,0.655155,1.031335,0.913670,0.750732,0.794630,1.053067,0.750740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084,2024,12,SEA,ARI,22,31,254.0,1,1.0,5.0,...,50.060001,72.059998,0.553571,1.109752,1.039754,0.842970,1.112544,0.707897,1.092261,1.112556
8085,2024,12,SF,GB,17,29,199.0,1,1.0,2.0,...,36.259998,53.260002,0.644444,0.775490,0.749297,0.859285,0.847372,1.022942,0.951560,0.847381
8086,2024,12,TB,NYG,24,30,294.0,0,0.0,0.0,...,78.760002,102.760002,0.483871,1.176605,1.031335,1.038756,1.069528,1.307376,0.989839,1.064235
8087,2024,12,TEN,HOU,18,25,278.0,2,1.0,8.0,...,74.119995,92.120003,0.431034,0.909195,0.913468,1.065949,0.921620,1.093094,0.935987,0.921630


In [47]:
opp_stats = [x + '_opp_roll' for x in cols2roll]
opp = per_game.copy()
opp[opp_stats] = opp.groupby("opponent_team")[
    cols2roll
].transform(lambda x: x.rolling(7, min_periods=1).mean()).sort_values(by='completions')
opp['yards_per_carry_opp_roll'] = opp['rushing_yards_opp_roll'] / opp['carries_opp_roll']
opp_stats.append('yards_per_carry_opp_roll')
opp[opp_stats] = opp[opp_stats] / opp[opp_stats].mean()
opp

,season,week,recent_team,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,fantasy_points_ppr,pass_pct,completions_opp_roll,attempts_opp_roll,carries_opp_roll,passing_yards_opp_roll,rushing_yards_opp_roll,pass_pct_opp_roll,receiving_yards_opp_roll,yards_per_carry_opp_roll
0,2010,1,ARI,LA,22,41,297.0,1,0.0,2.0,...,82.779999,0.661290,1.029544,1.208111,0.799576,1.225189,1.000146,1.173897,1.225201,1.255557
1,2010,1,ATL,PIT,27,44,252.0,0,1.0,2.0,...,66.080002,0.637681,1.263531,1.296509,0.951877,1.039554,0.517933,1.131986,1.039565,0.546167
2,2010,1,BAL,NYJ,20,38,248.0,0,1.0,2.0,...,59.619999,0.520548,0.935949,1.119713,1.332627,1.023053,0.437564,0.924056,1.023064,0.329584
3,2010,1,BUF,MIA,18,34,139.0,1,0.0,3.0,...,52.459999,0.666667,0.842354,1.001848,0.647276,0.573405,0.446494,1.183440,0.573411,0.692403
4,2010,1,CAR,NYG,14,35,182.0,1,3.0,4.0,...,50.380001,0.593220,0.655164,1.031314,0.913802,0.750789,0.794759,1.053061,0.750797,0.873005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084,2024,12,SEA,ARI,22,31,254.0,1,1.0,5.0,...,72.059998,0.553571,1.083027,1.056571,0.957316,1.005963,0.961875,1.050060,1.005973,1.008548
8085,2024,12,SF,GB,17,29,199.0,1,1.0,2.0,...,53.260002,0.644444,0.935949,0.892403,1.028027,0.850383,1.024384,0.937726,0.850392,1.000211
8086,2024,12,TB,NYG,24,30,294.0,0,0.0,0.0,...,102.760002,0.483871,0.875781,0.791376,1.136813,0.891636,1.469603,0.857493,0.891645,1.297610
8087,2024,12,TEN,HOU,18,25,278.0,2,1.0,8.0,...,92.120003,0.431034,0.949320,1.014476,0.957316,1.016571,0.912123,1.016691,1.016581,0.956382


In [48]:
opp.loc[opp.opponent_team=='KC',['season','week','recent_team','opponent_team','rushing_yards_opp_roll', 'yards_per_carry_opp_roll']]

,season,week,recent_team,opponent_team,rushing_yards_opp_roll,yards_per_carry_opp_roll
17,2010,1,LAC,KC,0.973357,0.884843
39,2010,2,CLE,KC,0.812619,0.779016
92,2010,3,SF,KC,0.669741,0.756698
139,2010,5,IND,KC,0.718855,0.750537
163,2010,6,HOU,KC,0.810833,0.868938
...,...,...,...,...,...,...
7963,2024,8,LV,KC,0.734801,0.831903
8004,2024,9,TB,KC,0.619989,0.762751
8015,2024,10,DEN,KC,0.625091,0.763936
8037,2024,11,BUF,KC,0.653157,0.767729


In [49]:
team[['recent_team','season','week'] + roll_stats].to_csv('data/agg/team_stats.csv')
opp[['opponent_team','season','week'] + opp_stats].to_csv('data/agg/opp_stats.csv')

In [50]:
opp.loc[opp.opponent_team=='CAR',['opponent_team','season','week'] + opp_stats]

,opponent_team,season,week,completions_opp_roll,attempts_opp_roll,carries_opp_roll,passing_yards_opp_roll,rushing_yards_opp_roll,pass_pct_opp_roll,receiving_yards_opp_roll,yards_per_carry_opp_roll
23,CAR,2010,1,0.935949,0.883984,1.370702,1.084931,1.053725,0.806891,1.084943,0.771645
61,CAR,2010,2,0.748759,0.810318,1.332627,0.909610,0.951032,0.779539,0.909619,0.716340
70,CAR,2010,3,0.795557,0.903628,1.358011,0.874545,0.991216,0.815553,0.874555,0.732653
116,CAR,2010,4,0.982746,1.031314,1.294552,0.939517,1.013541,0.888313,0.939527,0.785877
131,CAR,2010,5,0.861073,0.937023,1.355472,0.793691,1.200175,0.821234,0.793700,0.888765
...,...,...,...,...,...,...,...,...,...,...,...
7944,CAR,2024,7,0.942634,0.837680,1.316309,0.941727,1.447916,0.809245,0.941737,1.104125
7954,CAR,2024,8,1.016173,0.900821,1.289113,1.009499,1.348411,0.848769,1.009509,1.049940
7999,CAR,2024,9,1.042915,0.951335,1.272795,1.071966,1.320346,0.880685,1.071978,1.041268
8027,CAR,2024,10,1.009488,0.938706,1.370702,1.009499,1.463224,0.831701,1.009509,1.071521
